In [ ]:
#####################################################################################################
##  Uncomment following lines when running in Google Colab, to install the required dependencies.  ##
#####################################################################################################

import torch
assert torch.cuda.is_available(), 'You need a GPU! In Colab, go to Runtime -> Change runtime type -> Hardware accelerator -> GPU'

!pip install segments-ai
!pip install pyyaml==5.1
!pip install torch==1.7
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
#!git clone https://github.com/segments-ai/fast-labeling-workflow
%cd fast-labeling-workflow

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
/content/fast-labeling-workflow


# Speed up your image segmentation workflow with model-assisted labeling

A large dataset of labeled images is the first thing you need in any serious computer vision project.
Building such datasets is a time-consuming endeavour, involving lots of manual labeling work. This is especially true for tasks like image segmentation where the labels need to be very precise.

One way to drastically speed up image labeling is by leveraging your machine learning models from the start. 
Instead of labeling the entire dataset manually, you can use your model to help you by iterating between image labeling and model training.

This tutorial will show you how to achieve such a fast labeling workflow for image segmentation with Segments.ai.

![A fast labeling workflow](https://github.com/segments-ai/fast-labeling-workflow/blob/master/img/fast-labeling-workflow-diagram.png?raw=1)

[Segments.ai](https://segments.ai) is a labeling platform with powerful automation tools for image segmentation. 
It also features a flexible API and Python SDK, which enable you to quickly set up custom workflows by uploading images and labels directly from your code.

We will walk you through a simple but efficient setup:

1. Upload your images to Segments.ai, and label a small subset.
2. Train a segmentation model on the labeled images.
3. Generate label predictions on the remaining images and upload them.
4. Correct the mistakes.

You can find all code for this tutorial on [Github](https://github.com/segments-ai/fast-labeling-workflow), or follow along on [Google Colab](https://colab.research.google.com/github/segments-ai/fast-labeling-workflow/blob/master/demo.ipynb).

## 1. Upload your images and label a small subset

First, we need some images to label.

If you have a folder of images on your pc, you can simply upload them to Segments.ai through the web interface: first create a new dataset, then upload the samples.

But let's assume your data is in the cloud, and all you have is a list of image URLs. In this case, you can upload them to Segments.ai using our API or Python SDK. You need an API key for this, which can be created on your [account page](https://segments.ai/account).

In this tutorial, our goal is to label a dataset of 100 tomato images. First, we upload the images using the Python SDK:

In [ ]:
from segments import SegmentsClient # Install this package with 'pip install segments-ai'
from utils import get_image_urls

# Set up the client
client = SegmentsClient('1d506cc26a24a7e68c64e9600416cd0529adf6a9')
dataset_name = 'kailkuhn/playground' # Name of a dataset you've created on Segments.ai


** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



Initialized successfully.


Once the images are uploaded, click the "Start labeling" button on the samples tab of your dataset and get to work! Rather than immediately labeling the entire dataset, let's start out by labeling around 20 images.

Segments.ai's deep learning fueled superpixel tool makes the labeling a breeze.

## 2. Train a segmentation model on the labeled images

After you've labeled a few images, go to the releases tab of your dataset and create a new release, for example with the name "v0.1". A release is a snapshot of your dataset at a particular point in time.

Through the Python SDK, we can now initialize a SegmentsDataset from this release and visualize the labeled images. The SegmentsDataset is compatible with popular frameworks like PyTorch, Tensorflow and Keras.

In [ ]:
from segments import SegmentsDataset
from utils import visualize, train_model

# Initialize a dataset from the release file
release = client.get_release(dataset_name, 'v0.6')
dataset = SegmentsDataset(release, task='ground-truth', filter_by=['labeled'])

# Visualize a few samples in the dataset
#for sample in dataset:
#    print(sample['name'])    
#    visualize(sample['image'], sample['segmentation_bitmap']) 

  0%|          | 0/124 [00:00<?, ?it/s]

Initializing dataset. This may take a few seconds...


100%|██████████| 124/124 [00:14<00:00,  8.43it/s]

Initialized dataset with 124 images.


In [ ]:
len(dataset)

124

Next, let's train a computer vision model on the labeled images. Here we use Facebook's Detectron2 framework to train the model, but you can just as easily plug in your own custom models and training code.

In [ ]:
# Train an instance segmentation model on the dataset
from utils import train_model
model = train_model(dataset)

100%|██████████| 124/124 [00:08<00:00, 14.34it/s]


Exported to 7a689518-0b6d-4284-b13e-9a9e8db39ee8_coco.json.
Metadata(evaluator_type='coco', image_root='segments/kailkuhn_playground/v0.6', json_file='7a689518-0b6d-4284-b13e-9a9e8db39ee8_coco.json', name='my_dataset', thing_classes=['leaf'])
[12/22 23:47:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
   

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[12/22 23:47:21 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.6/dist-packages/detectron2/data/detection_utils.py:414: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/usr/local/lib/python3.6/dist-packages/detectron2/data/detection_utils.py:414: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. T

[12/22 23:47:29 d2.utils.events]:  eta: 0:01:51  iter: 19  total_loss: 3.162  loss_cls: 0.6788  loss_box_reg: 0.839  loss_mask: 0.6974  loss_rpn_cls: 0.8333  loss_rpn_loc: 0.08664  time: 0.3932  data_time: 0.0287  lr: 4.9953e-06  max_mem: 2432M
[12/22 23:47:37 d2.utils.events]:  eta: 0:01:42  iter: 39  total_loss: 2.898  loss_cls: 0.6632  loss_box_reg: 0.829  loss_mask: 0.6922  loss_rpn_cls: 0.636  loss_rpn_loc: 0.07994  time: 0.3902  data_time: 0.0110  lr: 9.9902e-06  max_mem: 2432M
[12/22 23:47:45 d2.utils.events]:  eta: 0:01:34  iter: 59  total_loss: 2.59  loss_cls: 0.6192  loss_box_reg: 0.8915  loss_mask: 0.6774  loss_rpn_cls: 0.3207  loss_rpn_loc: 0.07049  time: 0.3909  data_time: 0.0134  lr: 1.4985e-05  max_mem: 2558M
[12/22 23:47:53 d2.utils.events]:  eta: 0:01:26  iter: 79  total_loss: 2.348  loss_cls: 0.5701  loss_box_reg: 0.892  loss_mask: 0.6531  loss_rpn_cls: 0.1693  loss_rpn_loc: 0.0663  time: 0.3919  data_time: 0.0090  lr: 1.998e-05  max_mem: 2558M
[12/22 23:48:01 d2.util

## 3. Generate and upload label predictions for the unlabeled images

When the model is trained, we can run it on the unlabeled images to generate label predictions, and upload these predictions to Segments.ai:

In [ ]:
from segments.utils import bitmap2file

# Initialize a new dataset, this time containing only unlabeled images
dataset = SegmentsDataset(release, task='ground-truth', filter_by='prelabeled')

for sample in dataset:
    # Generate label predictions
    image = sample['image']
    segmentation_bitmap, annotations = model(image)
    
    # Visualize the predictions
    visualize(image, segmentation_bitmap)
    print(annotations)
    
    # Upload the predictions to Segments.ai
    file = bitmap2file(segmentation_bitmap)
    asset = client.upload_asset(file, 'label.png')    
    attributes = {
        'format_version': '0.1',
        'annotations': annotations,
        'segmentation_bitmap': { 'url': asset['url'] },
    }
    client.add_label(sample['uuid'], 'ground-truth', attributes, label_status='PRELABELED')

Output hidden; open in https://colab.research.google.com to view.

## 4. Verify and correct the predicted labels

Now go back to Segments.ai and click the "Start labeling" button again to continue labeling. This time, your job is quite a bit easier: instead of having to label each image from scratch, you can simply correct the few mistakes your model made!

The superpixel technology makes it very easy to correct the mistakes, and is a real time-saver here.

## Next steps

As you keep iterating between model training and labeling in this manner, your model will quickly get better and better. You'll reach a point where you're mostly just verifying the model's predictions, only having to correct the occasional mistakes on hard edge cases.

Was this useful for you? Let us know! Make sure to check out the Segments.ai [documentation](https://docs.segments.ai/python-sdk) and don't hesitate to [contact us](https://segments.ai/contact) if you have any questions.